In [1]:
import openai
import pandas as pd
from collections import Counter
import os

/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
'''
preprocess
 1.delete all rows include 'None'
 2.combine 
'''
jobs = pd.read_csv('processed_jobs.csv')
# print(Counter(jobs['adjusted_job']))

# jobs = jobs[jobs['detail'].notna()]
# jobs = jobs[jobs['adjusted_job'].notna()]
# jobs = jobs[jobs['job'].notna()]
# jobs = jobs[jobs['经验要求'].notna()]
# jobs = jobs[jobs['学历'].notna()]
# jobs = jobs[jobs['工资'].notna()]

# jobs.to_csv('processed_jobs.csv', index=False)

job_counts = jobs.groupby('adjusted_job').size()
jobs_merged = jobs.groupby('adjusted_job').agg({
    'detail': ' '.join,
    '工资': ' '.join,
    '学历': ' '.join,
    '经验要求': ' '.join
}).reset_index()

jobs_merged = jobs_merged.rename(columns={
    '工资': 'salary',
    '学历': 'edu_bg',
    '经验要求': 'expe',
})
print(jobs_merged.columns)

Index(['adjusted_job', 'detail', 'salary', 'edu_bg', 'expe'], dtype='object')


In [4]:
print(jobs_merged['adjusted_job'])

0           产品经理
1           人事专员
2           人力资源
3       供应链主管/经理
4           咨询顾问
5             客服
6           市场营销
7           广告设计
8     总裁/总经理/CEO
9           文科教师
10            法务
11          物流专员
12           管培生
13            编辑
14       行政助理/文员
15           运营岗
16          销售人员
17          项目管理
Name: adjusted_job, dtype: object


In [17]:
openai.api_key = "sk-nGwRkt4DhrQNpM1X01Af6b17E53b41FbB7E2B082E6C5B6D5"

# all client options can be configured just like the `OpenAI` instantiation counterpart
openai.base_url = "https://free.gpt.ge/v1/"
openai.default_headers = {"x-foo": "true"}

def extract_job_details(adjusted_job, detail, edu_bg, expe):
    prompt = f"""你是一位经验丰富的HR专家。请根据以下信息，为每个指标总结提炼要求，并直接输出，每个要求用逗号分隔，不带提示词：
专业知识与技能,
学习能力,
通识性知识与技能,
沟通与表达能力,
创新能力,
执行能力,
自我管理能力,
职业适应能力,
动手操作能力,
承压抗挫能力,
团队协作能力,
组织与管理能力

**职业**: "{adjusted_job}"  
**职位描述**: "{detail}"  
**学历要求**: "{edu_bg}"  
**经验要求**: "{expe}"
"""
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content



# 初始化空列表存储结果
results = []
results_abnor = []

jobs_unique = set(jobs_merged['adjusted_job'])


# 循环处理每条职位描述
for job_info in jobs_unique:
    job_title = job_info
    job_description = jobs_merged[jobs_merged['adjusted_job']==job_title]['detail']
    edu_background = jobs_merged[jobs_merged['adjusted_job']==job_title]['edu_bg']
    experience = jobs_merged[jobs_merged['adjusted_job']==job_title]['expe']
  
    result = extract_job_details(job_title, job_description, edu_background, experience)
    result_line = result.split(',')
    if(len(result_line)!=12):
        results_abnor.append({
            "工作岗位":job_title,
            "结果":result
        })
        continue

    major = result_line[0]
    study = result_line[1]
    genral_know = result_line[2]
    communi = result_line[3]
    innova = result_line[4]
    execute =result_line[5]
    self_manage = result_line[6]
    vocat_adjust = result_line[7]
    man_operat = result_line[8]
    pres_resis = result_line[9]
    team_opera = result_line[10]
    organ_manag = result_line[11]
    results.append({
    "工作岗位":job_title,
    "专业知识与技能":major,
    "学习能力":study,
    "通识性知识与技能":genral_know,
    "沟通与表达能力":communi,
    "创新能力":innova,
    "执行能力":execute,
    "自我管理能力":self_manage,
    "职业适应能力":vocat_adjust,
    "动手操作能力":man_operat,
    "承压抗挫能力":pres_resis,
    "团队协作能力":team_opera,
    "组织与管理能力":organ_manag,
    })

print(results)
print(results_abnor)

[{'工作岗位': '项目管理', '专业知识与技能': '项目管理专家要求:\n专业知识与技能: 精通项目管理理论和方法，具备项目管理相关知识和技能', '学习能力': '\n学习能力: 不断学习项目管理最新发展，不断提升项目管理技能', '通识性知识与技能': '\n通识性知识与技能: 具备跨学科知识，能够综合运用各方面知识解决问题', '沟通与表达能力': '\n沟通与表达能力: 能清晰有效地和团队沟通，具备良好的表达能力', '创新能力': '\n创新能力: 具备创新意识，能够在项目管理中提出新的思路和方法', '执行能力': '\n执行能力: 能够有效执行项目计划，推动项目进展，确保项目顺利完成', '自我管理能力': '\n自我管理能力: 能够有效管理自己的时间和工作，保持高效率', '职业适应能力': '\n职业适应能力: 能够适应项目管理领域的变化和挑战，灵活应对各种情况', '动手操作能力': '\n动手操作能力: 具备操作项目管理工具和软件的能力，能够实际操作项目管理工作', '承压抗挫能力': '\n承压抗挫能力: 具备承受项目压力和挫折的能力，能够在困难情况下保持积极态度', '团队协作能力': '\n团队协作能力: 能够与团队成员有效合作，共同推动项目进展', '组织与管理能力': '\n组织与管理能力: 具备组织规划和管理团队的能力，能够有效指导团队达成项目目标.'}, {'工作岗位': '管培生', '专业知识与技能': '熟悉招聘渠道和方法', '学习能力': '具备有效面试技巧', '通识性知识与技能': '了解人力资源管理相关知识', '沟通与表达能力': '良好的书面和口头表达能力', '创新能力': '具备创新思维能力', '执行能力': '具备组织和执行能力', '自我管理能力': '能有效管理时间和任务', '职业适应能力': '具备适应不同工作环境的能力', '动手操作能力': '具备基本办公软件操作技能', '承压抗挫能力': '能应对工作压力和挫折', '团队协作能力': '良好的团队合作精神', '组织与管理能力': '具备基本组织和管理能力'}]
[{'工作岗位': '咨询顾问', '结果': '专业知识与技能: 咨询研究, 客户服务, 沟通能力, 研究工作协助,\n学习能力: 快速学习新知识和技能,\n通识性知

In [1]:
results = [{'工作岗位': '项目管理', '专业知识与技能': '项目管理专家要求:\n专业知识与技能: 精通项目管理理论和方法，具备项目管理相关知识和技能', '学习能力': '\n学习能力: 不断学习项目管理最新发展，不断提升项目管理技能', '通识性知识与技能': '\n通识性知识与技能: 具备跨学科知识，能够综合运用各方面知识解决问题', '沟通与表达能力': '\n沟通与表达能力: 能清晰有效地和团队沟通，具备良好的表达能力', '创新能力': '\n创新能力: 具备创新意识，能够在项目管理中提出新的思路和方法', '执行能力': '\n执行能力: 能够有效执行项目计划，推动项目进展，确保项目顺利完成', '自我管理能力': '\n自我管理能力: 能够有效管理自己的时间和工作，保持高效率', '职业适应能力': '\n职业适应能力: 能够适应项目管理领域的变化和挑战，灵活应对各种情况', '动手操作能力': '\n动手操作能力: 具备操作项目管理工具和软件的能力，能够实际操作项目管理工作', '承压抗挫能力': '\n承压抗挫能力: 具备承受项目压力和挫折的能力，能够在困难情况下保持积极态度', '团队协作能力': '\n团队协作能力: 能够与团队成员有效合作，共同推动项目进展', '组织与管理能力': '\n组织与管理能力: 具备组织规划和管理团队的能力，能够有效指导团队达成项目目标.'}, 
           {'工作岗位': '管培生', '专业知识与技能': '熟悉招聘渠道和方法', '学习能力': '具备有效面试技巧', '通识性知识与技能': '了解人力资源管理相关知识', '沟通与表达能力': '良好的书面和口头表达能力', '创新能力': '具备创新思维能力', '执行能力': '具备组织和执行能力', '自我管理能力': '能有效管理时间和任务', '职业适应能力': '具备适应不同工作环境的能力', '动手操作能力': '具备基本办公软件操作技能', '承压抗挫能力': '能应对工作压力和挫折', '团队协作能力': '良好的团队合作精神', '组织与管理能力': '具备基本组织和管理能力'}]
results_abnor = [{'工作岗位': '咨询顾问', '结果': '专业知识与技能: 咨询研究, 客户服务, 沟通能力, 研究工作协助,\n学习能力: 快速学习新知识和技能,\n通识性知识与技能: 具备广泛的知识背景,\n沟通与表达能力: 准确表达观点, 有效沟通能力,\n创新能力: 提出创新想法和解决方案,\n执行能力: 计划有效实施, 独立完成工作,\n自我管理能力: 良好的时间和任务管理能力,\n职业适应能力: 快速适应新工作环境,\n动手操作能力: 熟练操作相关工具和设备,\n承压抗挫能力: 能在压力下保持良好状态,\n团队协作能力: 能有效地与团队合作,\n组织与管理能力: 有效组织和管理工作任务。'}, 
                 {'工作岗位': '文科教师', '结果': '专业知识与技能: 教育学专业知识，教学设计和评估技能，课堂管理技能，学科专业知识\n学习能力: 能够及时学习和掌握新的教学方法和理论知识\n通识性知识与技能: 学科以外的综合素质知识，如艺术、体育、心理学等\n沟通与表达能力: 具备良好的口头和书面表达能力，能够与学生和家长有效沟通\n创新能力: 具备在教学中创新的思维和方法，能够激发学生的创造力\n执行能力: 能够积极落实学校的教学计划和管理规定\n自我管理能力: 能够有效组织和管理个人教学和课余活动\n职业适应能力: 能够适应学校的教学环境和文化氛围\n动手操作能力: 能够灵活运用教学辅助工具和设备\n承压抗挫能力: 能够承受教学压力，对挫折和困难有适当的调整和应对能力\n团队协作能力: 能够积极参与学校教研和教学团队的合作和交流\n组织与管理能力: 能够有效组织和管理学生的学习活动和课外实践'}, {'工作岗位': '法务', '结果': '专业知识与技能: 法律知识, 业务准入评估技能, 法律立项评估技能, 合同管理技能, 诉讼处理技能\n学习能力: 能够学习新的法律法规和管理知识\n通识性知识与技能: 综合知识运用能力, 了解相关行业政策法规\n沟通与表达能力: 良好的口头和书面沟通能力, 能够准确表达法律观点和建议\n创新能力: 寻找解决问题的创新方法和策略\n执行能力: 能够按照公司制定的法律政策和流程执行工作\n自我管理能力: 能够独立工作并合理安排工作时间, 能够管理和处理多个项目和任务\n职业适应能力: 能够适应法律行业的变化和需求\n动手操作能力: 熟练操作法律相关软件和工具\n承压抗挫能力: 能够在工作压力下保持冷静和高效\n团队协作能力: 能够与团队成员合作并有效推动项目进展\n组织与管理能力: 能够组织和管理法律事务和项目的执行和进展'}, {'工作岗位': '人力资源', '结果': '专业知识与技能: 抗压能力, 人力资源管理, 沟通技巧, 组织能力\n\n学习能力: 学习能力强\n\n通识性知识与技能: 无要求\n\n沟通与表达能力: 沟通能力强, 表达能力强\n\n创新能力: 创新能力强\n\n执行能力: 执行能力强\n\n自我管理能力: 自我管理能力强\n\n职业适应能力: 职业适应能力强\n\n动手操作能力: 动手操作能力强\n\n承压抗挫能力: 抗压能力强\n\n团队协作能力: 团队协作能力强\n\n组织与管理能力: 组织能力强'}, {'工作岗位': '供应链主管/经理', '结果': '专业知识与技能: 供应链管理知识,仓储管理知识,运输配送知识  \n学习能力: 快速学习新的供应链业务模式和技术  \n通识性知识与技能: 商务谈判能力,数据分析能力  \n沟通与表达能力: 能清晰表达业务需求和方案,与不同团队有效沟通  \n创新能力: 提供创新的供应链解决方案  \n执行能力: 能有效执行供应链产品的业务模式设计  \n自我管理能力: 能有效管理自己的工作进度和任务  \n职业适应能力: 能适应不断变化的供应链市场和技术环境  \n动手操作能力: 能实际操作和管理仓储和运输设备  \n承压抗挫能力: 能承受工作压力,在挫折面前保持积极态度  \n团队协作能力: 能与团队协作,推动供应链业务发展  \n组织与管理能力: 能有效组织和管理供应链产品的业务模式设计项目。'}, {'工作岗位': '广告设计', '结果': '专业知识与技能: kv等创意设计, ppt美化, 长图文排版, 手册, 卡片, 包装设计排版, 简单手绘技能\n学习能力: 能够学习并掌握新的设计软件和技术\n通识性知识与技能: 基本的美术和设计原理知识\n沟通与表达能力: 能够与团队和客户有效沟通并表达设计理念和要求\n创新能力: 能够提供创新的设计方案和理念\n执行能力: 能够按时完成设计任务并具有高质量的设计执行能力\n自我管理能力: 能够有效地管理时间和任务，有良好的自我组织能力\n职业适应能力: 能够适应广告设计领域的快速变化和压力\n动手操作能力: 具备熟练的设计软件和工具操作能力\n承压抗挫能力: 能够在工作中承受工作压力和面对挫折，并积极寻找解决方案\n团队协作能力: 能够与团队成员协作，共同完成设计项目\n组织与管理能力: 能够组织和管理设计项目，保证项目的顺利进行'}, {'工作岗位': '物流专员', '结果': '专业知识与技能: 运营分析处理能力, 出口随机文件准备能力\n学习能力: 适应学习新的系统信息录入\n通识性知识与技能: 无\n沟通与表达能力: 无\n创新能力: 无\n执行能力: 进出口运单填制能力, 系统信息录入能力\n自我管理能力: 无\n职业适应能力: 无\n动手操作能力: 无\n承压抗挫能力: 处理时效, 滞留, 成本分析能力, 对工作压力的承受能力\n团队协作能力: 无\n组织与管理能力: 无'}, {'工作岗位': '人事专员', '结果': '专业知识与技能: 与物流、仓储、运输相关知识掌握, 熟悉仓储、运输业务职能要求\n学习能力: 能够快速学习物流、仓储、运输方面的知识\n通识性知识与技能: 具备一定的通用知识和技能\n沟通与表达能力: 能够与相关部门进行良好的沟通和表达\n创新能力: 具备在工作中提出创新解决方案的能力\n执行能力: 能够有效地执行工作任务\n自我管理能力: 具备自我规划和自我管理的能力\n职业适应能力: 能够迅速适应新的工作环境和岗位要求\n动手操作能力: 具备处理物流、仓储、运输相关操作的能力\n承压抗挫能力: 能够在工作压力下保持良好的心理状态和工作表现\n团队协作能力: 能够与团队成员协作，完成工作任务\n组织与管理能力: 具备组织和管理工作的能力'}, {'工作岗位': '总裁/总经理/CEO', '结果': '专业知识与技能: 项目策划、财务管理、城市发展、市场营销  \n学习能力: 快速掌握新领域知识  \n通识性知识与技能: 经济学、管理学、法律法规  \n沟通与表达能力: 清晰表达意见、能有效沟通  \n创新能力: 提出创新解决方案  \n执行能力: 能有效执行工作计划  \n自我管理能力: 时间管理、目标设定、自律  \n职业适应能力: 对不同领域快速适应  \n动手操作能力: 熟练操作办公软件、项目管理工具  \n承压抗挫能力: 处理工作压力、挫折能力强  \n团队协作能力: 能够与团队成员协作、协调工作  \n组织与管理能力: 有效组织资源、管理团队'}, {'工作岗位': '客服', '结果': '专业知识与技能: 客服流程和技巧, 问题解决能力, 电话沟通技巧  \n学习能力: 快速学习新产品和服务知识  \n通识性知识与技能: 社交礼仪和礼貌用语  \n沟通与表达能力: 清晰表达想法, 听取他人反馈  \n创新能力: 提出改善客户体验的新想法  \n执行能力: 能够快速有效地解决客户问题  \n自我管理能力: 组织时间和任务管理  \n职业适应能力: 能够应对客户各种不同情况和需求  \n动手操作能力: 熟练使用客服系统和工具  \n承压抗挫能力: 在高压情况下保持耐心和冷静  \n团队协作能力: 与团队合作解决客户问题  \n组织与管理能力: 能够有效处理客户投诉和问题管理'}, {'工作岗位': '市场营销', '结果': '专业知识与技能, 市场营销领域的活动策划和执行能力,现代市场营销的主流趋势的了解,\n学习能力, 不断学习和适应市场新趋势，活动策划和落地的新方法和技能,\n通识性知识与技能, 对城市特点和痛点的理解，现代市场营销的基础知识,\n沟通与表达能力, 有效沟通和表达活动策划的想法和计划,\n创新能力, 制定相符合城市特点及痛点的活动政策，不断结合现代主流动态,\n执行能力, 监督并落实活动政策，确保活动顺利落地,\n自我管理能力, 对活动策划和执行的自我管理和时间规划能力,\n职业适应能力, 能够适应市场营销领域的变化和挑战,\n动手操作能力, 在活动策划和执行中能够实际动手完成任务,\n承压抗挫能力, 能够在压力下坚持完成活动策划和执行任务,\n团队协作能力, 能够与团队合作实现活动政策的落地，组织与管理能力, 对活动的组织和管理能力。'}, {'工作岗位': '运营岗', '结果': '掌握财务管理、商业政策、合同管理、流程处理，\n学习能力强，\n具备通识性知识和技能，\n具有良好的沟通和表达能力，\n具备创新能力，\n具备执行能力，\n具备自我管理能力，\n具备职业适应能力，\n具备动手操作能力，\n具备承压抗挫能力，\n具备团队协作能力，\n具备组织与管理能力'}, {'工作岗位': '编辑', '结果': '专业知识与技能: 编辑知识和技能, 市场调研, 选题策划, 组织发掘作者和内容, 寻找配图资源和画手等\n学习能力: 学习市场调研方法, 学习选题策划技巧, 学习编辑相关知识和技能等\n通识性知识与技能: 通识知识, 通识技能等\n沟通与表达能力: 与作者和团队有效沟通和表达能力, 书面和口头表达能力等\n创新能力: 提出新的市场调研和选题策划思路, 创造性解决问题等\n执行能力: 完成市场调研和选题策划任务, 落实作者和内容的发掘等\n自我管理能力: 在规定的时间内完成工作, 自我组织和协调能力等\n职业适应能力: 适应编辑工作的要求, 快速掌握新领域知识和技能等\n动手操作能力: 使用编辑软件, 寻找配图资源等\n承压抗挫能力: 在工作压力下保持积极和抗挫能力等\n团队协作能力: 与团队成员合作开展工作, 协调团队合作等\n组织与管理能力: 组织市场调研和选题策划工作, 协调资源和时间管理等'}, {'工作岗位': '行政助理/文员', '结果': '专业知识与技能: 数据统计与分析, 资料搜集与整理\n学习能力: 快速学习并掌握新知识\n通识性知识与技能: 办公室行政知识\n沟通与表达能力: 良好的沟通和表达能力\n创新能力: 提出优化和改进的创新思路\n执行能力: 能够按时完成指定任务\n自我管理能力: 能够高效地管理工作时间和任务\n职业适应能力: 能够适应工作环境和职责变化\n动手操作能力: 熟练使用办公软件和设备\n承压抗挫能力: 能够在压力下保持良好的工作状态\n团队协作能力: 能够积极与团队合作，并达成共同目标\n组织与管理能力: 能够组织并管理办公室行政工作'}, {'工作岗位': '销售人员', '结果': '专业知识与技能:微信沟通技巧，客户咨询技能，产品知识\n学习能力:快速学习新产品知识，不断学习销售技巧和市场动态\n通识性知识与技能:销售技巧，市场分析，客户服务\n沟通与表达能力:清晰有效的沟通，良好的口头和书面表达能力\n创新能力:提供创新的销售方法和解决方案\n执行能力:有效地执行销售计划和任务，达成销售目标\n自我管理能力:良好的时间管理和自我激励能力\n职业适应能力:适应不同的销售环境和客户需求\n动手操作能力:熟练操作销售工具和设备\n承压抗挫能力:在高压和挫折的情况下保持积极态度和目标导向\n团队协作能力:与团队成员合作，共同达成销售目标\n组织与管理能力:有效组织销售活动和客户信息管理。'}, {'工作岗位': '产品经理', '结果': '专业知识与技能: 税务业务知识，产品规划和设计能力, 竞品分析能力,\n学习能力: 快速学习新的产品知识和技能,\n通识性知识与技能: 财税相关知识, 行业背景知识,\n沟通与表达能力: 能够清晰有效地沟通和表达需求和设计想法,\n创新能力: 提出新颖的产品规划和设计方案,\n执行能力: 能够有效地执行产品规划和设计方案,\n自我管理能力: 能够有效地管理自己的工作进度和任务优先级,\n职业适应能力: 能够迅速适应新的工作环境和需求变化,\n动手操作能力: 具备操作相关软件和工具的能力,\n承压抗挫能力: 能够在工作压力下保持稳定的心态和高效的工作状态,\n团队协作能力: 能够与跨部门团队协作，共同推动产品实施,\n组织与管理能力: 具备组织和管理产品规划和设计的能力。'}]

In [3]:
# 去掉换行符并调整格式
for entry in results:
    for key in entry:
        entry[key] = entry[key].replace('\n', '').strip()

# 输出合并后的列表
print(results)

[{'工作岗位': '项目管理', '专业知识与技能': '项目管理专家要求:专业知识与技能: 精通项目管理理论和方法，具备项目管理相关知识和技能', '学习能力': '学习能力: 不断学习项目管理最新发展，不断提升项目管理技能', '通识性知识与技能': '通识性知识与技能: 具备跨学科知识，能够综合运用各方面知识解决问题', '沟通与表达能力': '沟通与表达能力: 能清晰有效地和团队沟通，具备良好的表达能力', '创新能力': '创新能力: 具备创新意识，能够在项目管理中提出新的思路和方法', '执行能力': '执行能力: 能够有效执行项目计划，推动项目进展，确保项目顺利完成', '自我管理能力': '自我管理能力: 能够有效管理自己的时间和工作，保持高效率', '职业适应能力': '职业适应能力: 能够适应项目管理领域的变化和挑战，灵活应对各种情况', '动手操作能力': '动手操作能力: 具备操作项目管理工具和软件的能力，能够实际操作项目管理工作', '承压抗挫能力': '承压抗挫能力: 具备承受项目压力和挫折的能力，能够在困难情况下保持积极态度', '团队协作能力': '团队协作能力: 能够与团队成员有效合作，共同推动项目进展', '组织与管理能力': '组织与管理能力: 具备组织规划和管理团队的能力，能够有效指导团队达成项目目标.'}, {'工作岗位': '管培生', '专业知识与技能': '熟悉招聘渠道和方法', '学习能力': '具备有效面试技巧', '通识性知识与技能': '了解人力资源管理相关知识', '沟通与表达能力': '良好的书面和口头表达能力', '创新能力': '具备创新思维能力', '执行能力': '具备组织和执行能力', '自我管理能力': '能有效管理时间和任务', '职业适应能力': '具备适应不同工作环境的能力', '动手操作能力': '具备基本办公软件操作技能', '承压抗挫能力': '能应对工作压力和挫折', '团队协作能力': '良好的团队合作精神', '组织与管理能力': '具备基本组织和管理能力'}, {'工作岗位': '咨询顾问', '专业知识与技能': '专业知识与技能: 咨询研究, 客户服务, 沟通能力, 研究工作协助,', '学习能力': '快速学习新知识和技能,', '通识性知识与技能': '具备广泛的知

In [7]:
results_abnor = [{'工作岗位': '项目管理', '专业知识与技能': '精通项目管理理论和方法，具备项目管理相关知识和技能', '学习能力': '不断学习项目管理最新发展，不断提升项目管理技能', '通识性知识与技能': '具备跨学科知识，能够综合运用各方面知识解决问题', '沟通与表达能力': '能清晰有效地和团队沟通，具备良好的表达能力', '创新能力': '具备创新意识，能够在项目管理中提出新的思路和方法', '执行能力': '能够有效执行项目计划，推动项目进展，确保项目顺利完成', '自我管理能力': '能够有效管理自己的时间和工作，保持高效率', '职业适应能力': '能够适应项目管理领域的变化和挑战，灵活应对各种情况', '动手操作能力': '具备操作项目管理工具和软件的能力，能够实际操作项目管理工作', '承压抗挫能力': '具备承受项目压力和挫折的能力，能够在困难情况下保持积极态度', '团队协作能力': '能够与团队成员有效合作，共同推动项目进展', '组织与管理能力': '具备组织规划和管理团队的能力，能够有效指导团队达成项目目标.'}, 
 {'工作岗位': '管培生', '专业知识与技能': '熟悉招聘渠道和方法', '学习能力': '具备有效面试技巧', '通识性知识与技能': '了解人力资源管理相关知识', '沟通与表达能力': '良好的书面和口头表达能力', '创新能力': '具备创新思维能力', '执行能力': '具备组织和执行能力', '自我管理能力': '能有效管理时间和任务', '职业适应能力': '具备适应不同工作环境的能力', '动手操作能力': '具备基本办公软件操作技能', '承压抗挫能力': '能应对工作压力和挫折', '团队协作能力': '良好的团队合作精神', '组织与管理能力': '具备基本组织和管理能力'}, 
 {'工作岗位': '咨询顾问', '专业知识与技能': '咨询研究, 客户服务, 沟通能力, 研究工作协助,', '学习能力': '快速学习新知识和技能,', '通识性知识与技能': '具备广泛的知识背景,', '沟通与表达能力': '准确表达观点, 有效沟通能力,', '创新能力': '提出创新想法和解决方案,', '执行能力': '计划有效实施, 独立完成工作,', '自我管理能力': '良好的时间和任务管理能力,', '职业适应能力': '快速适应新工作环境,', '动手操作能力': '熟练操作相关工具和设备,', '承压抗挫能力': '能在压力下保持良好状态,', '团队协作能力': '能有效地与团队合作,', '组织与管理能力': '有效组织和管理工作任务。'}, 
 {'工作岗位': '文科教师', '专业知识与技能': '教育学专业知识，教学设计和评估技能，课堂管理技能，学科专业知识', '学习能力': '能够及时学习和掌握新的教学方法和理论知识', '通识性知识与技能': '学科以外的综合素质知识，如艺术、体育、心理学等', '沟通与表达能力': '具备良好的口头和书面表达能力，能够与学生和家长有效沟通', '创新能力': '具备在教学中创新的思维和方法，能够激发学生的创造力', '执行能力': '能够积极落实学校的教学计划和管理规定', '自我管理能力': '能够有效组织和管理个人教学和课余活动', '职业适应能力': '能够适应学校的教学环境和文化氛围', '动手操作能力': '能够灵活运用教学辅助工具和设备', '承压抗挫能力': '能够承受教学压力，对挫折和困难有适当的调整和应对能力', '团队协作能力': '能够积极参与学校教研和教学团队的合作和交流', '组织与管理能力': '能够有效组织和管理学生的学习活动和课外实践'}, 
 {'工作岗位': '法务', '专业知识与技能': '法律知识, 业务准入评估技能, 法律立项评估技能, 合同管理技能, 诉讼处理技能', '学习能力': '能够学习新的法律法规和管理知识', '通识性知识与技能': '综合知识运用能力, 了解相关行业政策法规', '沟通与表达能力': '良好的口头和书面沟通能力, 能够准确表达法律观点和建议', '创新能力': '寻找解决问题的创新方法和策略', '执行能力': '能够按照公司制定的法律政策和流程执行工作', '自我管理能力': '能够独立工作并合理安排工作时间, 能够管理和处理多个项目和任务', '职业适应能力': '能够适应法律行业的变化和需求', '动手操作能力': '熟练操作法律相关软件和工具', '承压抗挫能力': '能够在工作压力下保持冷静和高效', '团队协作能力': '能够与团队成员合作并有效推动项目进展', '组织与管理能力': '能够组织和管理法律事务和项目的执行和进展'},
 {'工作岗位': '人力资源', '专业知识与技能': '抗压能力, 人力资源管理, 沟通技巧, 组织能力', '学习能力': '学习能力强', '通识性知识与技能': '无要求', '沟通与表达能力': '沟通能力强, 表达能力强', '创新能力': '创新能力强', '执行能力': '执行能力强', '自我管理能力': '自我管理能力强', '职业适应能力': '职业适应能力强', '动手操作能力': '动手操作能力强', '承压抗挫能力': '抗压能力强', '团队协作能力': '团队协作能力强', '组织与管理能力': '组织能力强'}, 
 {'工作岗位': '供应链主管/经理', '专业知识与技能': '供应链管理知识,仓储管理知识,运输配送知识', '学习能力': '快速学习新的供应链业务模式和技术', '通识性知识与技能': '商务谈判能力,数据分析能力', '沟通与表达能力': '能清晰表达业务需求和方案,与不同团队有效沟通', '创新能力': '提供创新的供应链解决方案', '执行能力': '能有效执行供应链产品的业务模式设计', '自我管理能力': '能有效管理自己的工作进度和任务', '职业适应能力': '能适应不断变化的供应链市场和技术环境', '动手操作能力': '能实际操作和管理仓储和运输设备', '承压抗挫能力': '能承受工作压力,在挫折面前保持积极态度', '团队协作能力': '能与团队协作,推动供应链业务发展', '组织与管理能力': '能有效组织和管理供应链产品的业务模式设计项目。'}, 
 {'工作岗位': '广告设计', '专业知识与技能': 'kv等创意设计, ppt美化, 长图文排版, 手册, 卡片, 包装设计排版, 简单手绘技能', '学习能力': '能够学习并掌握新的设计软件和技术', '通识性知识与技能': '基本的美术和设计原理知识', '沟通与表达能力': '能够与团队和客户有效沟通并表达设计理念和要求', '创新能力': '能够提供创新的设计方案和理念', '执行能力': '能够按时完成设计任务并具有高质量的设计执行能力', '自我管理能力': '能够有效地管理时间和任务，有良好的自我组织能力', '职业适应能力': '能够适应广告设计领域的快速变化和压力', '动手操作能力': '具备熟练的设计软件和工具操作能力', '承压抗挫能力': '能够在工作中承受工作压力和面对挫折，并积极寻找解决方案', '团队协作能力': '能够与团队成员协作，共同完成设计项目', '组织与管理能力': '能够组织和管理设计项目，保证项目的顺利进行'}, 
 {'工作岗位': '物流专员', '专业知识与技能': '运营分析处理能力, 出口随机文件准备能力', '学习能力': '适应学习新的系统信息录入', '通识性知识与技能': '无', '沟通与表达能力': '无', '创新能力': '无', '执行能力': '进出口运单填制能力, 系统信息录入能力', '自我管理能力': '无', '职业适应能力': '无', '动手操作能力': '无', '承压抗挫能力': '处理时效, 滞留, 成本分析能力, 对工作压力的承受能力', '团队协作能力': '无', '组织与管理能力': '无'}, 
 {'工作岗位': '人事专员', '专业知识与技能': '与物流、仓储、运输相关知识掌握, 熟悉仓储、运输业务职能要求', '学习能力': '能够快速学习物流、仓储、运输方面的知识', '通识性知识与技能': '具备一定的通用知识和技能', '沟通与表达能力': '能够与相关部门进行良好的沟通和表达', '创新能力': '具备在工作中提出创新解决方案的能力', '执行能力': '能够有效地执行工作任务', '自我管理能力': '具备自我规划和自我管理的能力', '职业适应能力': '能够迅速适应新的工作环境和岗位要求', '动手操作能力': '具备处理物流、仓储、运输相关操作的能力', '承压抗挫能力': '能够在工作压力下保持良好的心理状态和工作表现', '团队协作能力': '能够与团队成员协作，完成工作任务', '组织与管理能力': '具备组织和管理工作的能力'}, 
 {'工作岗位': '总裁/总经理/CEO', '专业知识与技能': '项目策划、财务管理、城市发展、市场营销', '学习能力': '快速掌握新领域知识', '通识性知识与技能': '经济学、管理学、法律法规', '沟通与表达能力': '清晰表达意见、能有效沟通', '创新能力': '提出创新解决方案', '执行能力': '能有效执行工作计划', '自我管理能力': '时间管理、目标设定、自律', '职业适应能力': '对不同领域快速适应', '动手操作能力': '熟练操作办公软件、项目管理工具', '承压抗挫能力': '处理工作压力、挫折能力强', '团队协作能力': '能够与团队成员协作、协调工作', '组织与管理能力': '有效组织资源、管理团队'}, 
 {'工作岗位': '客服', '专业知识与技能': '客服流程和技巧, 问题解决能力, 电话沟通技巧', '学习能力': '快速学习新产品和服务知识', '通识性知识与技能': '社交礼仪和礼貌用语', '沟通与表达能力': '清晰表达想法, 听取他人反馈', '创新能力': '提出改善客户体验的新想法', '执行能力': '能够快速有效地解决客户问题', '自我管理能力': '组织时间和任务管理', '职业适应能力': '能够应对客户各种不同情况和需求', '动手操作能力': '熟练使用客服系统和工具', '承压抗挫能力': '在高压情况下保持耐心和冷静', '团队协作能力': '与团队合作解决客户问题', '组织与管理能力': '能够有效处理客户投诉和问题管理'}]
results.extend(results_abnor)

In [9]:
import pandas as pd
df =  pd.DataFrame(results)

In [10]:
df.to_csv('gpt-3.5-turbo-summary.csv',encoding='utf-8')